# B1 interactive filtering

In [1]:
cd qMRLab
startup

    startup at line 2 column 1
    startup at line 2 column 1
loading struct
loading io
loading statistics
loading optim
loading image


In [2]:
cmd = ['curl -L -o b1_map.zip https://osf.io/mw3sq/download/'];
[STATUS,MESSAGE] = unix(cmd);
unzip('b1_map.zip');

In [3]:
fn = fullfile('ResultsBank');
if ~exist(fn, 'dir')
    mkdir(fn);
end

In [4]:
clear all

Model = b1_dam;

% Format data structure so that they may be fit by the model
data = struct();
data.SFalpha = double(load_nii_data('SFalpha.nii.gz')); %load data
data.SF2alpha  = double(load_nii_data('SF2alpha.nii.gz'));
data.Mask=double(load_nii_data(['..' filesep 'SF60-MASK.nii.gz']));


method_types = {'DAM', 'Method2', 'Method3'};
filtering_types = {'spline', 'polynomial', 'gaussian', 'median'};
filtering_dimensions = {'2D', '3D'};
filtering_orders = {2, 4, 6};
size_methods = {'gaussian', 'median'};

# octave_compatible = {'spline', 'polynomial'};
# Checks if it's octave
# cacheval = (exist ("OCTAVE_VERSION", "builtin") > 0)

for mm= 1:length(method_types)
    method_type= method_types{mm}
    fn = fullfile('ResultsBank', method_type);
    if ~exist(fn, 'dir')
        mkdir(fn);
    end
end

destination_folder_path = ['ResultsBank' filesep 'DAM'];
for i= 1:length(filtering_types)
    filter_type= filtering_types{i};
    for j = 1: length(filtering_dimensions)
        dimension = filtering_dimensions{j};
        for k = 1: length(filtering_orders)
            order = filtering_orders{k};
            
            # adjust model for filtering
            Model.options.Smoothingfilter_Type = filter_type;
            Model.options.Smoothingfilter_Dimension = dimension;
            
            if sum(strcmp(size_methods, filter_type))>0
                Model.options.Smoothingfilter_sizex = order;
                Model.options.Smoothingfilter_sizey = order;
                Model.options.Smoothingfilter_sizez = order;
            else 
                Model.options.Smoothingfilter_order = order;
            end
            
            try 
                FitResults = FitData(data,Model);
            catch
                continue;
            end
            
            # save fit results in correct location on disk
            B1_raw = FitResults.B1map_raw;
            B1_filtered = FitResults.B1map_filtered;
            typepath = fullfile(destination_folder_path, filter_type);
            mkdir(typepath, [dimension filesep num2str(order)]);
            try
                fulldestination = fullfile(typepath, [dimension filesep num2str(order) filesep 'B1_raw.mat']);
                save(fulldestination, 'B1_raw', '-v7');
                fulldestination = fullfile(typepath, [dimension filesep num2str(order) filesep 'B1_filtered.mat']);
                save(fulldestination, 'B1_filtered', '-v7');
            catch Message
                Message
                continue;
            end
            
        end
    end
end

ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
method_type = DAM
method_type = Method2
method_type = Method3
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.111977 seconds.
Operation has been completed: b1_dam
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.0929441 seconds.
Operation has been completed: b1_dam
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.108604 seconds.

In [5]:
import nibabel as nib

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_plotly_dash import JupyterDash

import plotly_express as px
import plotly.graph_objs as go

import matplotlib.pyplot as mpl
from matplotlib import cm
import numpy as np

from scipy.io import loadmat
import os

filesep = os.path.sep;


#  load first image for display on the final figure
SFalpha = nib.load('qMRLab' + filesep + 'SFalpha.nii.gz');
data_SFalpha = SFalpha.get_fdata();
fig_SFalpha = px.imshow(data_SFalpha);
fig_SFalpha.data[0].showscale=False;
fig_SFalpha.data[0].coloraxis=None;
fig_SFalpha.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50));
fig_SFalpha.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False);

#  load second image for display on the final figure
SF2alpha = nib.load('qMRLab' + filesep + 'SF2alpha.nii.gz');
data_SF2alpha = SF2alpha.get_fdata();
fig_SF2alpha = px.imshow(data_SF2alpha, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
fig_SF2alpha.data[0].showscale=False;
fig_SF2alpha.data[0].coloraxis=None;
fig_SF2alpha.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50));
fig_SF2alpha.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False);

dropdown_data = {'B1 method' : ['DAM', 'Method2', 'Method3'],
        'Filter type':['Spline', 'Polynomial', 'Gaussian', 'Median'],
        'Dimension':['2D', '3D'], 
        'Filtering order': [2,4,6]}

app = JupyterDash("b1_dash_app")

app.layout = html.Div(
    [
        html.H1("B1 double angle mapping - interactive filtering"),
        html.Div(
            [
                html.P([d + ":", dcc.Dropdown(id=d, options=[dict(label=m, value=m) for m in dropdown_data[d]], value=dropdown_data[d][0])], 
                      style={"width":"25%", "display":"inline-block"})
                for d in dropdown_data.keys()
            ],
            style={"width": "100%"},
        ),
        dcc.Graph(id="SFalpha", figure = fig_SFalpha, style={"width": "25%", "display": "inline-block"}),
         dcc.Graph(id="SF2alpha", figure = fig_SF2alpha, style={"width": "25%", "display": "inline-block"}),
        dcc.Graph(id="graph_raw", style={"width": "25%", "display": "inline-block"}),
        dcc.Graph(id="graph_filtered", style={"width": "25%", "display": "inline-block"}),
    ]
)


@app.callback([Output("graph_raw", "figure"), Output("graph_filtered", "figure")], [Input(d, "value") for d in dropdown_data.keys()])
def make_figure(method_type, filtering_type, dimension, order):

    path = '.' + filesep + 'qMRLab' + filesep + 'ResultsBank' + filesep + method_type + filesep + filtering_type.lower() + filesep + dimension + filesep + str(order) + filesep;
    file_path_raw = path  + 'B1_raw.mat';
    file_path_filtered = path + 'B1_filtered.mat';
    
    title_for_figure = "Type: %s, dimension: %s and order %s" % (filtering_type, dimension, order)
    B1_raw = loadmat(file_path_raw);
    b1_raw_data = B1_raw['B1_raw'];
    fig_raw = px.imshow(b1_raw_data, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
#     fig_raw.data[0].showscale=False
#     fig_raw.data[0].coloraxis=None
    fig_raw.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text= title_for_figure, coloraxis_showscale=False)
    fig_raw.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

    title_for_figure = "Type: %s, dimension: %s and order %s" % (filtering_type, dimension, order)
    B1_filtered = loadmat(file_path_filtered);
    b1_filtered_data = B1_filtered['B1_filtered'];
    fig_filtered = px.imshow(b1_filtered_data, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
#     fig_filtered.data[0].showscale=False
#     fig_filtered.data[0].coloraxis=None
    fig_filtered.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text= title_for_figure, coloraxis_showscale=False)
    fig_filtered.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
    
    return fig_raw, fig_filtered 

app

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
